# Distribution by counties

In [1]:
import geopandas as gpd
import rasterio
from rasterstats import zonal_stats
import os
import numpy as np
import fiona
from tqdm import tqdm
from shapely.geometry import shape, Polygon
from rasterio import features

# Table of contents



#### [1. Counties preparation](#Counties-preparation)

#### [2. Scenario 1](#Scenario-1)

#### [3. Scenario 2](#Scenario-2)

#### [4. Scenario 3](#Scenario-3)


# Counties preparation
[[Click here to turn back to the table of contents]](#Table-of-contents)

Loading base layer to see total number of pixels

In [4]:
#taking parameters from Estonian layer
path_scenario_1_r = '../../Thesis_2024_updates/Raster_Data/Scenario_1/'
file_name_whole_estonia_r = 'whole_estonia/Estonia_rasterized_0_1.tif'
raster_base_layer = os.path.join(path_scenario_1_r, file_name_whole_estonia_r)

with rasterio.open(raster_base_layer) as src:
    base_trans = src.transform
    base_profile = src.profile
    base_shape = src.shape
    base_data = src.read(1)
    
# Create a boolean mask where True represents pixels with value 1
mask = base_data == 1

# Count the number of True values in the mask
num_total_pixels = np.sum(mask)
total_area_ha = round((num_total_pixels * 16)/10000)

print("Number of pixels equal to 1:", num_total_pixels)
print("Total area of Estonia: {} ha".format(total_area_ha))

Number of pixels equal to 1: 2716698675
Total area of Estonia: 4346718 ha


Loading counties

In [5]:
#taking parameters from Estonian layer
#vector path (input)
file_name_counties = '../../Thesis_2024_updates/Vector_Data/counties/maakond_shp/maakond_20231101.shp'

Reading counties

In [6]:
# Load the shapefile with the counties
counties = gpd.read_file(file_name_counties)
counties.shape

(15, 3)

Preprocess counties

In [7]:
counties['name'] = counties['MNIMI'].apply(lambda x: x[:-8] + ' county')
counties.head()

,MNIMI,MKOOD,geometry,name
0,Saare maakond,0074,"MULTIPOLYGON (((455191.283 6404986.597, 455187...",Saare county
1,Viljandi maakond,0084,"MULTIPOLYGON (((621049.180 6453125.120, 621052...",Viljandi county
2,Hiiu maakond,0039,"MULTIPOLYGON (((418048.719 6506294.706, 418045...",Hiiu county
3,Harju maakond,0037,"MULTIPOLYGON (((504775.973 6570562.880, 504787...",Harju county
4,Lääne maakond,0056,"MULTIPOLYGON (((460628.287 6512871.566, 460630...",Lääne county


Calculating the areas of the counties

In [8]:
# Open the raster layer
with rasterio.open(raster_base_layer) as src:
    affine = src.transform
    array = src.read(1)  # Read the first band
    
    # Define the statistic to calculate: count of pixels with value 1
    stats = zonal_stats(counties, array, affine=affine, stats=['count'], 
                        categorical=True, nodata=src.nodata)

    # Adding the results to the counties GeoDataFrame
    counties['num_pixels_county'] = [s.get(1, 0) for s in stats]  # Get count of 1's, default to 0 if no 1's
    counties['county_area_ha'] = counties['num_pixels_county'].apply(lambda x: round((x * 16)/10000))

# Scenario 1
[[Click here to turn back to the table of contents]](#Table-of-contents)

Loading suitable lands

In [7]:
file_name_suitable_lands_s1 = '../../Thesis_2024_updates/Subtraction_Results/Step27_no_electric_lines.tif'

In [8]:
# Open the raster layer
with rasterio.open(file_name_suitable_lands_s1) as src:
    affine = src.transform
    array = src.read(1)  # Read the first band
    
    # Define the statistic to calculate: count of pixels with value 1
    stats = zonal_stats(counties, array, affine=affine, stats=['count'], 
                        categorical=True, nodata=src.nodata)

    # Adding the results to the counties GeoDataFrame
    counties['num_pixels_S1'] = [s.get(1, 0) for s in stats]  # Get count of 1's, default to 0 if no 1's
    counties['S1_suitable_area_ha'] = counties['num_pixels_S1'].apply(lambda x: round((x * 16)/10000))

In [9]:
counties['percent_of_county'] = round((counties['S1_suitable_area_ha']/counties['county_area_ha']) * 100, 1)
counties['percent_of_county_str'] = counties['percent_of_county'].apply(lambda x: str(x) + '%')

In [10]:
counties.head()

,MNIMI,MKOOD,geometry,name,num_pixels_county,county_area_ha,num_pixels_S1,S1_suitable_area_ha,percent_of_county,percent_of_county_str
0,Saare maakond,0074,"MULTIPOLYGON (((455191.283 6404986.597, 455187...",Saare county,183644942,293832,10701319,17122,5.8,5.8%
1,Viljandi maakond,0084,"MULTIPOLYGON (((621049.180 6453125.120, 621052...",Viljandi county,213756560,342010,5369366,8591,2.5,2.5%
2,Hiiu maakond,0039,"MULTIPOLYGON (((418048.719 6506294.706, 418045...",Hiiu county,64535957,103258,4046066,6474,6.3,6.3%
3,Harju maakond,0037,"MULTIPOLYGON (((504775.973 6570562.880, 504787...",Harju county,270490035,432784,15975309,25560,5.9,5.9%
4,Lääne maakond,0056,"MULTIPOLYGON (((460628.287 6512871.566, 460630...",Lääne county,113484461,181575,7689872,12304,6.8,6.8%


In [11]:
counties.to_file('../../Thesis_2024_updates/Vector_Data/counties/counties_with_pixel_count_S1.shp')

/var/folders/z4/9280v_g94039tvyzq2n4yrd40000gn/T/ipykernel_20631/556985071.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  counties.to_file('../../Thesis_2024_updates/Vector_Data/counties/counties_with_pixel_count_S1.shp')


In [13]:
counties_nice = counties.sort_values(by='percent_of_county', ascending=False)[['name', 'S1_suitable_area_ha', 'county_area_ha', 'percent_of_county_str']]
counties_nice

,name,S1_suitable_area_ha,county_area_ha,percent_of_county_str
4,Lääne county,12304,181575,6.8%
2,Hiiu county,6474,103258,6.3%
3,Harju county,25560,432784,5.9%
0,Saare county,17122,293832,5.8%
13,Lääne-Viru county,15001,369538,4.1%
6,Ida-Viru county,11844,297177,4.0%
8,Tartu county,12808,334947,3.8%
12,Pärnu county,20032,541910,3.7%
5,Rapla county,8491,276423,3.1%
10,Võru county,8191,277317,3.0%


# Scenario 2
[[Click here to turn back to the table of contents]](#Table-of-contents)

Loading suitable lands

In [ ]:
file_name_suitable_lands_s2 = '../../Thesis_2024_updates/Subtraction_Results/'

In [ ]:
# Open the raster layer
with rasterio.open(file_name_suitable_lands_s2) as src:
    affine = src.transform
    array = src.read(1)  # Read the first band
    
    # Define the statistic to calculate: count of pixels with value 1
    stats = zonal_stats(counties, array, affine=affine, stats=['count'], 
                        categorical=True, nodata=src.nodata)

    # Adding the results to the counties GeoDataFrame
    counties['num_pixels_S2'] = [s.get(1, 0) for s in stats]  # Get count of 1's, default to 0 if no 1's
    counties['S2_suitable_area_ha'] = counties['num_pixels_S2'].apply(lambda x: round((x * 16)/10000))

In [ ]:
counties['percent_of_county_S2'] = round((counties['S2_suitable_area_ha']/counties['county_area_ha']) * 100, 1)
counties['percent_of_county_S2_str'] = counties['percent_of_county_S2'].apply(lambda x: str(x) + '%')

# Scenario 3
[[Click here to turn back to the table of contents]](#Table-of-contents)

Loading suitable lands

In [14]:
file_name_suitable_lands_s3 = '../../Thesis_2024_updates/Subtraction_Results/SCENARIO_3_seminatural_grasslands.tif'

In [16]:
# Open the raster layer
with rasterio.open(file_name_suitable_lands_s3) as src:
    affine = src.transform
    array = src.read(1)  # Read the first band
    
    # Define the statistic to calculate: count of pixels with value 1
    stats = zonal_stats(counties, array, affine=affine, stats=['count'], 
                        categorical=True, nodata=src.nodata)

    # Adding the results to the counties GeoDataFrame
    counties['num_pixels_S3'] = [s.get(1, 0) for s in stats]  # Get count of 1's, default to 0 if no 1's
    counties['S3_suitable_area_ha'] = counties['num_pixels_S3'].apply(lambda x: round((x * 16)/10000))

In [23]:
counties['percent_of_county_S3'] = round((counties['S3_suitable_area_ha']/counties['county_area_ha']) * 100, 1)
counties['percent_of_county_S3_str'] = counties['percent_of_county_S3'].apply(lambda x: str(x) + '%')

In [24]:
S3_counties = counties[['name', 'county_area_ha', 'S3_suitable_area_ha', 'percent_of_county_S3', 'percent_of_county_S3_str', 'geometry']]

In [25]:
S3_counties.to_file('../../Thesis_2024_updates/Vector_Data/counties/counties_with_pixel_count_S3.shp')

/var/folders/z4/9280v_g94039tvyzq2n4yrd40000gn/T/ipykernel_20631/1407140751.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  S3_counties.to_file('../../Thesis_2024_updates/Vector_Data/counties/counties_with_pixel_count_S3.shp')


In [27]:
counties_nice_S3 = S3_counties.sort_values(by='percent_of_county_S3', ascending=False)[['name', 'S3_suitable_area_ha', 'county_area_ha', 'percent_of_county_S3_str']]
counties_nice_S3

,name,S3_suitable_area_ha,county_area_ha,percent_of_county_S3_str
3,Harju county,22299,432784,5.2%
6,Ida-Viru county,11357,297177,3.8%
8,Tartu county,12398,334947,3.7%
13,Lääne-Viru county,13600,369538,3.7%
4,Lääne county,5568,181575,3.1%
5,Rapla county,8131,276423,2.9%
0,Saare county,8181,293832,2.8%
10,Võru county,7496,277317,2.7%
14,Põlva county,4838,182337,2.7%
2,Hiiu county,2734,103258,2.6%


# Historical forest
[[Click here to turn back to the table of contents]](#Table-of-contents)

Loading suitable lands

In [2]:
file_name_historical_forest = '../../Thesis_2024_updates/Subtraction_Results/Step19_no_historical.tif'

In [9]:
# Open the raster layer
with rasterio.open(file_name_historical_forest) as src:
    affine = src.transform
    array = src.read(1)  # Read the first band
    
    # Define the statistic to calculate: count of pixels with value 1
    stats = zonal_stats(counties, array, affine=affine, stats=['count'], 
                        categorical=True, nodata=src.nodata)

    # Adding the results to the counties GeoDataFrame
    counties['num_pixels_HF'] = [s.get(1, 0) for s in stats]  # Get count of 1's, default to 0 if no 1's
    counties['HF_suitable_area_ha'] = counties['num_pixels_HF'].apply(lambda x: round((x * 16)/10000))

In [10]:
counties['percent_of_county_HF'] = round((counties['HF_suitable_area_ha']/counties['county_area_ha']) * 100, 1)
# counties['percent_of_county_HF_str'] = counties['percent_of_county_HF'].apply(lambda x: str(x) + '%')

In [11]:
HF_counties = counties[['name', 'county_area_ha', 'HF_suitable_area_ha', 'percent_of_county_HF', 'geometry']]
HF_counties

,name,county_area_ha,HF_suitable_area_ha,percent_of_county_HF,geometry
0,Saare county,293832,78171,26.6,"MULTIPOLYGON (((455191.283 6404986.597, 455187..."
1,Viljandi county,342010,106312,31.1,"MULTIPOLYGON (((621049.180 6453125.120, 621052..."
2,Hiiu county,103258,24071,23.3,"MULTIPOLYGON (((418048.719 6506294.706, 418045..."
3,Harju county,432784,121949,28.2,"MULTIPOLYGON (((504775.973 6570562.880, 504787..."
4,Lääne county,181575,59353,32.7,"MULTIPOLYGON (((460628.287 6512871.566, 460630..."
5,Rapla county,276423,91371,33.1,"POLYGON ((504136.450 6536681.570, 504465.710 6..."
6,Ida-Viru county,297177,63305,21.3,"MULTIPOLYGON (((714923.260 6544236.101, 714913..."
7,Valga county,191704,49901,26.0,"MULTIPOLYGON (((622138.806 6444214.901, 622139..."
8,Tartu county,334947,122161,36.5,"MULTIPOLYGON (((705484.908 6476994.745, 705479..."
9,Järva county,267412,96116,35.9,"POLYGON ((560449.290 6519256.000, 560687.830 6..."


In [13]:
HF_counties['Unsuitable_area_ha'] = HF_counties['county_area_ha'] - HF_counties['HF_suitable_area_ha']

/Users/rav/dev/virtual_env/lib/python3.9/site-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [14]:
HF_counties.Unsuitable_area_ha.sum()

3026201

In [15]:
3026201/4346718

0.6962036644659257